## Mountain Car Reinforcement Learning

For this project I decided to implement what we have learned about reinforcement learning on the Mountain Car environment from the OpenAI Gym/Gymnasium tool. The intent is to develop multiple Q-Learning algorithms and determine if I can get better Rewards from one over the others. 

The majority of the work will be completed in individual code files listed below in this document.

For the development of this project I worked locally on my personal machine. 

#### Mountain Car Environment

The Mountain Car MDP is a deterministic MDP where the car is placed at the bottom of a valley. The goal of the MDP is to accelerate the car to reach the goal state (the flag) at the top of the right hill. There are two versions of the mountain car, one with discrete actions and one with continuous. For the purposes of this project I will be working with the one with discrete actions.

The Mountain Car environment is part of the Classic Control environments. Its Action Space = Discrete(3) and it's Observation Space = Box([-1.2 -0.07], [0.6 0.07], (2,), float32).

The Action space includes three actions:
- O: Push the car to the left
- 1: Do nothing
- 2: Push the car to the right

The Observation Space contains positional and velocity elements:
- The first element is the position which can vary between -1.2 and 0.6, representing the car's horizontal position on the track
- The second element is the velocity which can vary between -0.07 and 0.07, representing the car's actual velocity

This all means that at each step the car must choose one of the three discrete actions and is in a current state given by the continuous vector (Observation Space)

More information about the environment can be found here: https://gymnasium.farama.org/environments/classic_control/mountain_car/ 